In [1]:
import torch

In [5]:
B,T,C = 4,8,32

In [6]:
x = torch.randn(B,T,C) # (B,T,C)
x.shape

torch.Size([4, 8, 32])

In [8]:
# x[0]

# # version 1

In [9]:
xbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1,:] # (t,C)
        xbow[b,t] = torch.mean(xprev, dim=0, keepdims=True)
xbow[0]

tensor([[-0.0585, -1.4621],
        [-0.0805, -1.2181],
        [-0.0773, -0.7803],
        [-0.2062, -0.6981],
        [ 0.0896, -0.7327],
        [ 0.0190, -0.4635],
        [-0.0247, -0.2036],
        [-0.1994, -0.1567]])

# # version 2

In [35]:
# toy example

a = torch.ones(3,3)
a = torch.tril(a) # makes the tensor a lower triangle matrix
a = a / torch.sum(a, dim=1, keepdim=True) # normalizing the lower triangle matrix to create averaging effect while information sharing
b = torch.randint(0,10, (3,2)).float()
c = a @ b

print(a)
print('----------')
print(b)
print('----------')
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----------
tensor([[7., 7.],
        [0., 0.],
        [0., 6.]])
----------
tensor([[7.0000, 7.0000],
        [3.5000, 3.5000],
        [2.3333, 4.3333]])


In [36]:
# doing the above in batched manner
# weighted aggregation - wei will be the weights to do the aggregation
wei = torch.tril(torch.ones(T,T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [37]:
xbow2 = wei @ x # (T,T) @ (B,T,C) --> (B,T,C)
xbow2[0]

tensor([[-0.0585, -1.4621],
        [-0.0805, -1.2181],
        [-0.0773, -0.7803],
        [-0.2062, -0.6981],
        [ 0.0896, -0.7327],
        [ 0.0190, -0.4635],
        [-0.0247, -0.2036],
        [-0.1994, -0.1567]])

In [38]:
torch.allclose(xbow, xbow2)

True

# # version 3

In [44]:
tril = torch.tril(torch.ones(T,T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [42]:
# using Softmax

wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf'))
wei = torch.nn.functional.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [43]:
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

# # version 4: self attention!

In [25]:
# a single head performing self-attention

head_size = 16
key = torch.nn.Linear(C, head_size, bias=False)
query = torch.nn.Linear(C, head_size, bias=False)
value = torch.nn.Linear(C, head_size, bias=False)

k = key(x)   # (B,T,head_size)
q = query(x) # (B,T,head_size)
v = value(x) # (B,T,head_size)
wei = q @ k.transpose(-2,-1) # (B,T,head_size) @ (B,head_size,T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T)) # (T,T)
# wei = torch.zeros(T,T)
wei = wei.masked_fill(tril==0, float('-inf')) # (B,T,T)
wei = torch.nn.functional.softmax(wei, dim=-1)

out = wei @ v # (B,T,T) @ (B,T,head_size) --> (B,T,head_size)
out.shape

torch.Size([4, 8, 16])

In [24]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5903, 0.4097, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1067, 0.5428, 0.3505, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3086, 0.5181, 0.1142, 0.0591, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2246, 0.3787, 0.0635, 0.1166, 0.2167, 0.0000, 0.0000, 0.0000],
        [0.1750, 0.0839, 0.1429, 0.1312, 0.0449, 0.4221, 0.0000, 0.0000],
        [0.0645, 0.0390, 0.0535, 0.0648, 0.7122, 0.0361, 0.0299, 0.0000],
        [0.0548, 0.1188, 0.1672, 0.0676, 0.0488, 0.5212, 0.0056, 0.0160]],
       grad_fn=<SelectBackward0>)